In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as func
from pyspark.sql.functions import countDistinct
from pyspark.sql.types import *
import re
from pyspark.sql.functions import *
sc.stop()
sc =SparkContext()
sqlContext = SQLContext(sc)


In [ ]:
year= [2017,2018,2019]
month = [12,1,2,3]
day = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
month_dict = {1: "January",2: "February",3 : "march",4 :"April",5 :"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"}

#read in category data
dat_iab= sqlContext.read.parquet("gs://bbm-data-warehouse-production-bigdata/tables/inventory-perf/")

#read,process all of the click data
for y in year:
    for m in month:
        for d in day:
            for key,value in  month_dict.iteritems():
                if key == m:
                    month_name = value 
            date = str(y)+'-'+str(m)+'-'+str(d)
            try:
                new_data_path = sqlContext.read.parquet("gs://ds-taste-dfp/aggregated_data/click_aggregated_by_date/year="+str(y)+'/'+month_name+'/'+date+'.parquet')
                print new_data_path
                print "New file already exists"
            except Exception,e:
                print "New file doesn't exists ="
                path = 'gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year='+str(y)+'/month='+str(m)+'/day='+str(d)+'/*/*'
                print path
                try:
                    original_file = sqlContext.read.parquet(path)
                    status =  "The path to create new file exists"
                except Exception,e:
                    status =  "The path to create new file doesn't exists"
                    print status
                if status == "The path to create new file exists":
                    print status
                    print path
                    #Get the first group of click data from flat logs 
                    click_data = original_file.select('orderId','placementid','tag','oppId','event_name','itemID').where(((col('tag') == "OFFER") | (col('tag') == "RENDER")))

                    #cleaning the data for filtering
                    click_data = click_data.withColumn('Status',when((col('orderId') == 'YAHOO') | (col('orderId') == 'INMOBI') | (col('orderId') == 'GOOGLE') | (col('orderId') == 'ANIVIEW') ,'True').otherwise('False'))
                    click_data = click_data.withColumn('Status_itemid',when((col('itemID') == 'C00000DFP') | (col('itemID') == 'C0000DFP') | (col('itemID') == 'GOOGLE') | (col('itemID') == 'C00ANIVIEW') | (col('itemID') == 'ANIVIEW') ,'True').otherwise('False'))


                    click_data_new = click_data.select('tag','orderId','itemID','oppId').where((col('Status') == 'False') & (col('Status_itemid') == 'False')).sort(desc("orderId"))

                    #Get the second group of events from flat logs 
                    event_name = original_file.select('event_name','requestid','adid').where((col('event_name') == "onAdRequest"))



                    #join the dataset
                    from pyspark.sql.functions import col

                    final_data = click_data_new.join(event_name,col('oppId')== col('requestid'),'left')
                    data_clicks = final_data.selectExpr("orderId as orderid","itemID as line_itemid","tag as type","adid")

                    # #processing click data
                    data_clicks= data_clicks.withColumn('Impressions',when(col('type') == 'RENDER',1).otherwise(0))
                    data_clicks = data_clicks.withColumn('Clicks',when(col('type') == 'OFFER',1).otherwise(0))
                    data_clicks_new = data_clicks.where(col('adid').isNotNull())
                    data_clicks_aggre_one = data_clicks_new.groupby('orderid','line_itemid','adid').agg(func.sum("Impressions").alias('impressions'),func.sum("Clicks").alias('clicks'))
                    dfp_click_iab = data_clicks_aggre_one.join(dat_iab,(dat_iab.order_id == data_clicks_aggre_one.orderid) & (dat_iab.line_item_id == data_clicks_aggre_one.line_itemid),'left' ).select([(xx) for xx in data_clicks_aggre_one.columns]+[dat_iab.iab_tier1])

                   
                    write_path ="gs://ds-taste-dfp/aggregated_data/click_aggregated_by_date/year="+str(y)+'/'+month_name+'/'+date+'.parquet'
                    dfp_click_iab.dropDuplicates().write.mode('overwrite').parquet(write_path)



New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=12/day=1/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=12/day=2/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=12/day=3/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=12/day=4/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=12/day=5/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=12/day=6/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-pr

The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=1/day=23/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=1/day=24/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=1/day=25/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=1/day=26/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=1/day=27/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=1/day=28/*/*
The path to create new file doesn't exists
New file 

The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=3/day=13/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=3/day=14/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=3/day=15/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=3/day=16/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=3/day=17/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2017/month=3/day=18/*/*
The path to create new file doesn't exists
New file 

DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressi

DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
DataFrame[orderid: string, line_itemid: string, adid: string, impressions: bigint, clicks: bigint, IABTier1Categorization: string]
New file already exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2018/month=2/day=29/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-bigdata/tables/flat-logs/year=2018/month=2/day=30/*/*
The path to create new file doesn't exists
New file doesn't exists =
gs://bbm-data-warehouse-production-big